In [2]:
#%% 赛题目的：预测单位销量(unit_sales) 书上第11章
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import swifter
import polars as pl
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from datetime import date,datetime,timedelta    

In [3]:
#%% 数据读取
path = '/Users/ziyan/Documents/机器学习/机器学习比赛/favorita-grocery-sales-forecasting/'
# 只用pandas
time1 = time.time()
df_train = pd.read_csv(path+'train.csv',parse_dates=['date'])
df_train['unit_sales'] = df_train['unit_sales'].astype(float).apply(lambda u:np.log1p(u) if u>0 else 0)
print('1:',time.time() - time1)

# 用polars读再转swifter ，节省一半时间 注意这部分代码可能要单独运行 有几率失败
time2 = time.time()
df_train = pl.read_csv(path+'train.csv',dtypes={'date': pl.Datetime,'onpromotion':bool,'unit_sales':float})
df_train = df_train.to_pandas()
df_train['unit_sales'] = df_train['unit_sales'].swifter.apply(lambda u:np.log1p(u) if u>0 else 0.0)
print('2:',time.time() - time2)

df_test = pd.read_csv(path+'test.csv',parse_dates=['date'])

items = pd.read_csv(path+'items.csv')
stores = pd.read_csv(path+'stores.csv')

df_train.isnull().sum() 
# onpromotion有21657651个缺失
df_train['onpromotion'].value_counts()
# False 96028767,True 7810622
z = df_test.describe()

1: 113.05473113059998


Dask Apply:   0%|          | 0/17 [00:00<?, ?it/s]

/Users/ziyan/opt/anaconda3/lib/python3.9/site-packages/dask/utils.py:1241: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/Users/ziyan/opt/anaconda3/lib/python3.9/site-packages/dask/utils.py:1241: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/Users/ziyan/opt/anaconda3/lib/python3.9/site-packages/dask/utils.py:1241: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)


2: 58.740147829055786


In [4]:
z

,id,date,store_nbr,item_nbr
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06
mean,1.271823e+08,2017-08-23 12:00:00.000003072,2.750000e+01,1.244798e+06
min,1.254970e+08,2017-08-16 00:00:00,1.000000e+00,9.699500e+04
25%,1.263397e+08,2017-08-19 18:00:00,1.400000e+01,8.053210e+05
50%,1.271823e+08,2017-08-23 12:00:00,2.750000e+01,1.294665e+06
75%,1.280249e+08,2017-08-27 06:00:00,4.100000e+01,1.730015e+06
max,1.288675e+08,2017-08-31 00:00:00,5.400000e+01,2.134244e+06
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05


In [5]:
#%% 数据准备 
# 只保留2017年的数据，利用久远的数据对未来预测会产生一定的噪声（两者不太相关）
df_2017 = df_train[df_train.date >= pd.Timestamp(2017,1,1)]
del df_train

print(df_2017.describe())
print(' ')
print(df_2017.isnull().sum())

                 id                        date     store_nbr      item_nbr  \
count  2.380826e+07                    23808261  2.380826e+07  2.380826e+07   
mean   1.135929e+08  2017-04-24 15:38:17.344120  2.794909e+01  1.160406e+06   
min    1.016888e+08         2017-01-01 00:00:00  1.000000e+00  9.699500e+04   
25%    1.076408e+08         2017-02-27 00:00:00  1.300000e+01  6.875490e+05   
50%    1.135929e+08         2017-04-25 00:00:00  2.800000e+01  1.176562e+06   
75%    1.195450e+08         2017-06-20 00:00:00  4.300000e+01  1.501544e+06   
max    1.254970e+08         2017-08-15 00:00:00  5.400000e+01  2.127114e+06   
std    6.872853e+06                         NaN  1.621998e+01  5.795109e+05   

         unit_sales  
count  2.380826e+07  
mean   1.698823e+00  
min    0.000000e+00  
25%    1.098612e+00  
50%    1.609438e+00  
75%    2.197225e+00  
max    9.749579e+00  
std    8.723154e-01  
 
id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onp

In [6]:
#%% 特征：train中出现的store_nbr✖️item_nbr的商品是否promo（test中新出现的不管
# 先转化为三维索引，再按照日期做unstack，每一列都是不同的日期 第一列2017-1-1，第二列2017-1-2，
# unstack(level=-1)把最后一维（日期）作为列 fillna(False) 把空值填为False
# 这样就处理成了按列走向的时间序列 尚不明确意义
promo_2017_train = df_2017.set_index(['store_nbr','item_nbr','date'])[['onpromotion']].unstack(level=-1).fillna(False)
#help(pd.MultiIndex.get_level_values)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

promo_2017_test = df_test.set_index(['store_nbr','item_nbr','date'])[['onpromotion']].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
# 强行把索引搞成一样，类似于用promo_2017_train来LEFTJOIN test（因为这是找train?）
promo_2017 = pd.concat([promo_2017_train,promo_2017_test],axis=1)
# axis=1 列取并集

In [7]:
#%% 标签：unit_sales
df_2017 = df_2017.set_index(['store_nbr','item_nbr','date'])[['unit_sales']].unstack(level=-1).fillna(0)
# fillna(0)是从实际出发，那一天没有记录就代表没有卖出东西
df_2017.columns = df_2017.columns.get_level_values(1)

In [8]:
#%% 特征提取
def get_date_range(df,dt,forward_steps,periods,freq='D'):
    '''
    Description
    ----------
    截取时间段的通用函数
    
    Parameters
    ----------
    df : pd.DataFrame
        按照日期在列的方式处理好的df.
    dt : datetime.date
        某一天，例如date(2017,2,28).
    forward_steps : int
        回溯的天数，注意这里的steps指天数.
    periods : int
        整个date_range一共含多少个日期.
    freq : str, optional
        步长，'D'表示1天，'7D'表示7天. The default is 'D'.

    Returns
    -------
    pd.DataFrame
        返回df对应日期序列的记录.

    '''
    return df[pd.date_range(start=dt-timedelta(days=forward_steps),
                            periods=periods,#periods是总个数
                            freq=freq)] #freq表示步长 

def prepare_dataset(t2017,is_train=True):
    X = pd.DataFrame({
        # 历史平移特征： t-1，t-2，t-3天的销量
        # 字典方式构造dataframe
        'day_1_hist':get_date_range(df_2017,t2017,1,1).values.ravel(),#按照行拉伸至1维
        'day_2_hist':get_date_range(df_2017,t2017,2,1).values.ravel(),#按照行拉伸至1维
        'day_3_hist':get_date_range(df_2017,t2017,3,1).values.ravel(),#按照行拉伸至1维        
    })

    for i in [7,14,21,30]:
        # 窗口统计特征，销量diff/mean/med/max/min/std
        # 注意下面的批量命名变量方式 diff(axis=1)是某一行中第t+1列元素减去第t列元素 mean(axis=1)是某一行中对每一列元素求均值
        # axis=1则最后行数不变 列数可能有变化
        # 「i天销量之差」的均值
        X['diff_{}_day_mean'.format(i)] = get_date_range(df_2017,t2017,i,i).diff(axis=1).mean(axis=1).values
        # i天销量的均值
        X['mean_{}_day'.format(i)] = get_date_range(df_2017,t2017,i,i).mean(axis=1).values
        # i天销量的中位数
        X['median_{}_day'.format(i)] = get_date_range(df_2017,t2017,i,i).median(axis=1).values
        # i天销量的最大值
        X['max_{}_day'.format(i)] = get_date_range(df_2017,t2017,i,i).max(axis=1).values
        # i天销量的最小值
        X['min_{}_day'.format(i)] = get_date_range(df_2017,t2017,i,i).min(axis=1).values
        # i天销量的标准差
        X['std_{}_day'.format(i)] = get_date_range(df_2017,t2017,i,i).std(axis=1).values

    for i in range(7):
        # 前4周（本周不算）每周第i+1天(i=0~6)的平均销量
        X['mean_4_dow{}_2017'.format(i)] = get_date_range(df_2017,t2017,28-i,4,freq='7D').mean(axis=1).values
        # 前10周（本周不算）每周第i+1天(i=0~6)的平均销量
        X['mean_10_dow{}_2017'.format(i)] = get_date_range(df_2017,t2017,70-i,10,freq='7D').mean(axis=1).values

    for i in range(16):
        # 未来16天是否为促销日（为什么是16天？）
        X['promo_{}'.format(i)] = promo_2017[str(t2017+timedelta(days=i))].values
        
    if is_train:
    # 如果是训练数据，由于df数据的列是日期，选择未来16天作为y_train
        y = df_2017[pd.date_range(t2017,periods=16)].values
        return X,y
    return X

In [9]:
#%% 划分样本集
X_l,y_l = [],[]
t2017 = date(2017,7,5)
n_range = 14
for i in tqdm(range(n_range)):
    delta = timedelta(days=7*i)
    X_tmp,y_tmp = prepare_dataset(t2017-delta)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
    
X_train = pd.concat(X_l,axis=0)
y_train = np.concatenate(y_l,axis=0)
del X_l,y_l
    
X_val,y_val = prepare_dataset(date(2017,7,26))
X_test = prepare_dataset(date(2017,8,16),is_train=False)

100%|███████████████████████████████████████████| 14/14 [00:13<00:00,  1.02it/s]


In [10]:
#%% 模型训练
params = {
    'num_leaves':2**5-1,
    'objective':'regression_l2',
    'max_depth':8,
    'min_data_in_leaf':50,
    'learning_rate':0.05,
    'feature_fraction':0.75,#列采样
    'bagging_fraction':0.75,#行采样
    'bagging_freq':1,#每1侈迭代就bagging一次
    'metric':'l2'
}
MAX_ROUNDS = 500
val_pred=[]
test_pred=[]

#递归使用one-step-ahead，每次将预测值作为真实值加入训练集
for i in range(16):
    print('====== Step %d ======'%(i+1))
    dtrain = lgb.Dataset(X_train,
                         label=y_train[:,i])#一次选一天当标签
    dval = lgb.Dataset(X_val,
                       label=y_val[:,i],#一次选一天当标签
                       reference=dtrain)#If this is Dataset for validation, training data should be used as reference.
    bst = lgb.train(params,dtrain,MAX_ROUNDS,
                    valid_sets=[dtrain,dval],verbose_eval=100)
    
    
    val_pred.append(bst.predict(X_val,
                               num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(X_test,
                                 num_iteration=bst.best_iteration or MAX_ROUNDS))

====== Step 1 ======
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.098092 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10487
[LightGBM] [Info] Number of data points in the train set: 2345210, number of used features: 57
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 1.018067
[100]	training's l2: 0.313668	valid_1's l2: 0.30201
[200]	training's l2: 0.309921	valid_1's l2: 0.299609
[300]	training's l2: 0.308087	valid_1's l2: 0.298714
[400]	training's l2